In [1]:
!pip install -q transformers ipywidgets jupyter tensorflow textualheatmap

distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None


In [2]:
import os 
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, TFBertForQuestionAnswering
import tensorflow as tf
from textualheatmap import TextualHeatmap
  
tokenizer = AutoTokenizer.from_pretrained("./bert_model/")
model = TFBertForQuestionAnswering.from_pretrained("./bert_model/", from_pt=True)

with open('./sample.txt') as f:
    context = f.read()[500:1500]
        
question = "Does the paitent feel numb?"
print(context)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForQuestionAnswering: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


ays 
the pain mainly goes down just to his calf and not really his foot he says occasionally goes to his foot but is mostly in his 
thigh and calf. aggravating factor activity alleviating factors are rest. he reports weakness in his right lower extremity as 
well. he says this occurs when he has flareups but currently does not have the weakness.. denies any numbness or 
tingling. he says had pain since 1975 in his back. he occasionally has flareups but these have gotten worse and worse. 
he is done physical therapy and this made him worse. he has been offered injections but is refused these. he is 
concerned about his diabetes. i have offered him injection several times. he does not wish to have any. 
physical examination: 
alert and oriented x 4. patient in no acute distress. normal affect and mentation. 
the patient stands with the shoulders and pelvis level. sagittal and frontal contours of the spine are well maintained. 
examination of the low back region reveals the skin to be int

In [3]:
with open('./sample.txt') as f:
    context = f.read()[500:1500]
        
question = "Does the paitent have diabetes?"
print(context)

ays 
the pain mainly goes down just to his calf and not really his foot he says occasionally goes to his foot but is mostly in his 
thigh and calf. aggravating factor activity alleviating factors are rest. he reports weakness in his right lower extremity as 
well. he says this occurs when he has flareups but currently does not have the weakness.. denies any numbness or 
tingling. he says had pain since 1975 in his back. he occasionally has flareups but these have gotten worse and worse. 
he is done physical therapy and this made him worse. he has been offered injections but is refused these. he is 
concerned about his diabetes. i have offered him injection several times. he does not wish to have any. 
physical examination: 
alert and oriented x 4. patient in no acute distress. normal affect and mentation. 
the patient stands with the shoulders and pelvis level. sagittal and frontal contours of the spine are well maintained. 
examination of the low back region reveals the skin to be int

In [4]:
def mask_scores(scores, best_idx, token_ids_size):
    """
    Unhappy this needs to be its own function
    """
    mask = np.zeros((1, token_ids_size))
    mask[0, best_idx] = 1
    mask_tensor = tf.constant(mask, dtype='float32')
    masked_scores = scores * mask_tensor
    return masked_scores
    

def get_gradient(question, context, model, tokenizer): 
    embedding_matrix = model.get_input_embeddings()
    encoded_tokens =  tokenizer.encode_plus(question, context, add_special_tokens=True, return_token_type_ids=True, return_tensors="tf")
    token_ids = list(encoded_tokens["input_ids"].numpy()[0])
    vocab_size = embedding_matrix.vocab_size

    # convert token ids to one hot. We can't differentiate wrt to int token ids hence the need for one hot representation
    token_ids_tensor = tf.constant([token_ids], dtype='int32')
    token_ids_tensor_one_hot = tf.one_hot(token_ids_tensor, vocab_size) 
    
    with tf.GradientTape() as tape:
        # (i) watch input variable
        tape.watch(token_ids_tensor_one_hot)

        # multiply input model embedding matrix; allows us do backprop wrt one hot input 
        inputs_embeds = tf.matmul(token_ids_tensor_one_hot, embedding_matrix.weights[0]) 
        
        # (ii) get prediction
        output = model({"inputs_embeds": inputs_embeds, "token_type_ids": encoded_tokens["token_type_ids"], "attention_mask": encoded_tokens["attention_mask"] })
        start_scores = output.start_logits
        end_scores = output.end_logits
        best_start = np.argmax(start_scores)
        best_end = np.argmax(end_scores) + 1
        
        mask_start_scores = mask_scores(start_scores, best_start, len(token_ids))
        mask_end_scores = mask_scores(end_scores, best_end, len(token_ids))
        
        gradients = tf.norm(tape.gradient([mask_start_scores, mask_end_scores], token_ids_tensor_one_hot), axis=2)
        
        token_words = tokenizer.convert_ids_to_tokens(token_ids)
        answer_text = tokenizer.decode(token_ids[best_start:best_end])
        token_types = list(encoded_tokens["token_type_ids"].numpy()[0])
        
        print(answer_text)
        
        return gradients.numpy()[0], token_words, answer_text, token_types

In [5]:
gradients, token_words, answer_text, token_types = get_gradient(question, context, model, tokenizer)

[CLS]


In [6]:
import matplotlib.pyplot as plt

In [7]:
grad_df = pd.DataFrame({'word' : token_words,
                        'gradient' : gradients,
                        'type' : token_types})

grad_df['gradient'] = grad_df['gradient'] / grad_df['gradient'].max()
grad_df

,word,gradient,type
0,[CLS],0.095414,0
1,Does,0.511312,0
2,the,0.105904,0
3,p,0.286545,0
4,##ait,1.000000,0
...,...,...,...
219,to,0.015552,1
220,be,0.012456,1
221,in,0.025764,1
222,##t,0.021678,1


In [8]:
question = grad_df[grad_df['type'] == 0]
answer = grad_df[grad_df['type'] == 1]
max_grad_idx = np.argmax(answer['gradient'].to_numpy())
focused = answer.iloc[max_grad_idx-50:max_grad_idx+50]

In [9]:
from IPython.core.display import display, HTML

def get_html_str(words, gradients):
    ret_str = ''
    for i in range(len(words)):
        word = words[i]
        grad = int(gradients[i] * 25 + 75) - 15
        grad_f = gradients[i] * 0.7 + 0.05
        ret_str += f'<span style="background-color: hsla(110, 70%, 50%, {grad_f});">{word}</span><span> </span>'
        
    return ret_str
        
heatmap = get_html_str(focused['word'].to_list(), focused['gradient'].to_list())
display(
    HTML(heatmap)
)

In [10]:
# only take gradient wrt correct output? DONE
# fix from_pt error? DONE
# better output? DONE
# clean tokens
# package all of this up into a .py?

## TESTING GRADIENTS.PY

In [11]:
import gradients as grad

In [12]:
tokenizer = AutoTokenizer.from_pretrained("./bert_model/")
model = TFBertForQuestionAnswering.from_pretrained("./bert_model/", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForQuestionAnswering: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [21]:
with open('./sample.txt') as f:
    context = f.read()[500:1500]
        
question = "Does the patient feel numb?"
print(context)

ays 
the pain mainly goes down just to his calf and not really his foot he says occasionally goes to his foot but is mostly in his 
thigh and calf. aggravating factor activity alleviating factors are rest. he reports weakness in his right lower extremity as 
well. he says this occurs when he has flareups but currently does not have the weakness.. denies any numbness or 
tingling. he says had pain since 1975 in his back. he occasionally has flareups but these have gotten worse and worse. 
he is done physical therapy and this made him worse. he has been offered injections but is refused these. he is 
concerned about his diabetes. i have offered him injection several times. he does not wish to have any. 
physical examination: 
alert and oriented x 4. patient in no acute distress. normal affect and mentation. 
the patient stands with the shoulders and pelvis level. sagittal and frontal contours of the spine are well maintained. 
examination of the low back region reveals the skin to be int

In [22]:
grad_df = grad.get_gradients(model, tokenizer, question, context)
grad_df

,word,gradient,type
0,[CLS],0.078237,0
1,Does,0.232568,0
2,the,0.073792,0
3,patient,0.202431,0
4,feel,0.352740,0
...,...,...,...
217,to,0.010792,1
218,be,0.012990,1
219,in,0.024540,1
220,##t,0.020170,1


In [23]:
text = grad.html(grad_df['word'].tolist(), grad_df['gradient'].tolist())

In [24]:
from IPython.core.display import display, HTML

display(HTML(text))